In [1]:
import pandas as pd
import os


filepath = os.path.join("lichess","games.csv")


In [2]:
games_df = pd.read_csv(filepath)
games_df.head(1)

In [4]:
opening_variation_s = games_df["opening_name"].str.split(": ", n=1, expand = True)

In [5]:
games_df["opening"] = opening_variation_s[0]
games_df["variation"] = opening_variation_s[1]
games_df["opening"].value_counts()

Sicilian Defense                                               2573
French Defense                                                 1306
Queen's Pawn Game                                              1059
Italian Game                                                    981
King's Pawn Game                                                917
                                                               ... 
King's Gambit Accepted |  Ghulam-Kassim Gambit                    1
Alekhine Defense #3                                               1
Blackmar-Diemer Gambit Declined |  Weinsbach Declination          1
Blackmar-Diemer Gambit Declined |  Brombacher Countergambit       1
Australian Defense                                                1
Name: opening, Length: 227, dtype: int64